# Part 4: Using GPU acceleration with PyTorch

In [0]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

## Manual use of `.cuda()`

Now the magic of PyTorch comes in. So far, we've only been using the CPU to do computation. When we want to scale to a bigger problem, that won't be feasible for very long.
|
PyTorch makes it really easy to use the GPU for accelerating computation. Consider the following code that computes the element-wise product of two large matrices:

In [0]:
import torch

t1 = torch.randn(1000, 1000)
t2 = torch.randn(1000, 1000)
t3 = t1*t2
print(t3)

tensor([[-1.6793e+00, -2.9559e-01, -4.8968e-01,  ..., -3.9029e-03,
         -4.0842e-01, -4.0237e-01],
        [ 2.1154e-01,  4.9688e-01,  1.1669e+00,  ...,  9.9797e-02,
          7.0266e-01,  2.5722e-01],
        [-2.4888e-02,  4.1645e-02,  9.1157e-05,  ..., -1.6866e-01,
          2.7082e-02,  2.0227e+00],
        ...,
        [ 1.3039e+00,  1.7574e-01, -1.6591e+00,  ...,  1.0014e+00,
          6.3418e-02, -1.2458e-01],
        [-2.4242e-01, -1.5288e+00,  1.1601e-02,  ..., -4.0701e-03,
          2.4037e+00,  1.9300e+00],
        [-1.1845e-01,  3.2909e-01, -4.6137e-01,  ..., -2.6884e+00,
         -3.3244e-01, -1.0814e-02]])


By sending all the tensors that we are using to the GPU, all the operations on them will also run on the GPU without having to change anything else. If you're running a non-cuda enabled version of PyTorch the following will throw an error; if you have cuda available the following will create the input matrices, copy them to the GPU and perform the multiplication on the GPU itself:

In [0]:
"""
t1 = torch.randn(1000, 1000).cuda()
t2 = torch.randn(1000, 1000).cuda()
t3 = t1*t2
print(t3)

"""

RuntimeError: ignored

If you're running this workbook in colab, now enable GPU acceleration (`Runtime->Runtime Type` and add a `GPU` in the hardware accelerator pull-down). You'll then need to re-run all cells to this point.

If you were able to run the above with hardware acceleration, the print-out of the result tensor would show that it was an instance of `cuda.FloatTensor` type on the the `(GPU 0)` GPU device. If your wanted to copy the tensor back to the CPU, you would use the `.cpu()` method.

## Writing platform agnostic code

Most of the time you'd like to write code that is device agnostic; that is it will run on a GPU if one is available, and otherwise it would fall back to the CPU. The recommended way to do this is as follows:

In [0]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
t1 = torch.randn(1000, 1000).to(device)
t2 = torch.randn(1000, 1000).to(device)
t3 = t1*t2
print(t3)

tensor([[ 1.5385e-02, -1.4885e+00, -6.7286e-01,  ...,  5.0845e-01,
         -3.7193e-02,  2.3315e-01],
        [-8.3571e-01,  1.6217e-02, -1.1099e+00,  ..., -1.0349e-01,
         -5.3808e-01,  1.1004e-01],
        [-1.2449e+00, -8.5894e-01,  1.5156e-01,  ...,  2.7532e-01,
          5.2966e-03, -1.7810e-01],
        ...,
        [-5.6132e-02, -5.3309e-01,  4.0723e+00,  ..., -1.0339e-01,
         -9.7826e-01,  2.8635e-03],
        [-1.2540e-01, -4.4742e-02, -5.9271e-01,  ..., -4.5317e-02,
         -6.5087e-01,  2.4590e-02],
        [ 4.1656e+00, -6.3344e-01,  2.3524e-01,  ..., -2.9286e-01,
          1.1541e-02,  5.5555e-01]])


## Accelerating neural net training

If you wanted to accelerate the training of a neural net using raw PyTorch, you would have to copy both the model and the training data to the GPU. Unless you were using a really small dataset like MNIST, you would typically _stream_ the batches of training data to the GPU as you used them in the training loop:

```python
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = BaselineModel(784, 784, 10).to(device)

loss_function = ...
optimiser = ...

for epoch in range(10):
    for data in trainloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimiser.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimiser.step()
```

Using Torchbearer, this becomes much simpler - you just tell the `Trial` to run on the GPU and that's it!:

```python
model = BetterCNN()

loss_function = ...
optimiser = ...

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(trainloader)
trial.run(epochs=10)
```


## Multiple GPUs

Using multiple GPUs is beyond the scope of the lab, but if you have multiple cuda devices, they can be referred to by index: `cuda:0`, `cuda:1`, `cuda:2`, etc. You have to be careful not to mix operations on different devices, and would need how to carefully orchestrate moving of data between the devices (which can really slow down your code to the point at which using the CPU would actually be faster).

## Questions

__Answer the following questions (enter the answer in the box below each one):__

__1.__ What features of GPUs allow them to perform computations faster than a typically CPU?

1. A graphical processing unit (GPU), has smaller-sized but many more logical cores (arithmetic logic units or ALUs, control units and memory cache) whose basic design is to process a set of simpler and more identical computations in parallel.
A CPU has few complex cores

2. GPUs specialize in parallel processing using several concurrent hardware threads
CPU uses single thread performance optimization
A CPU (the brain) can work on a variety of different calculations, while a GPU (the brawn) is best at focusing all the computing abilities on a specific task. That is because a CPU consists of a few cores (up to 24) optimized for sequential serial processing. It is designed to maximize the performance of a single task within a job; however, the range of tasks is wide.

3. GPUs also maximize floating point throughput. CPU allocates a transistor space exclusively for complex tasks/computation.

__2.__ What is the biggest limiting factor for training large models with current generation GPUs?

Memory.

We can estimate the amount of memory required before training a model. For example, training AlexNet with batch size of 128 requires 1.1GB of global memory, and that is just 5 convolutional layers plus 2 fully-connected layers. If we look at a bigger model, say VGG-16, using a batch size of 128 will require about 14GB of global memory. The current state-of-the-art NVIDIA Titan X has a memory capacity of 12GB . 

To train a large model:

– reduce your batch size, which might hinder both your training speed and accuracy.

– distribute your model among multiple GPU(s), which is a complicated process in itself.

– reduce your model size, if you find yourself unwilling to do the aforementioned options , or you have already tried these options but they’re not good.

This implies a trade-off between time and space.